# Analyzing monochromator filter sweeps.
Craig Lage 19-Dec-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pickle as pkl

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/calib/unbounded"])
best_effort_isr = BestEffortIsr(embargo=True)
best_effort_isr.doWrite = False  # Don't write to butler database

# This cell gets the vendor supplied photodiode response.

In [ ]:
import csv
pd_response = {}
with open('/home/c/cslage/u/AuxTel/filter_data/hamamatsu_responsivity-1.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    counter = 0
    for row in spamreader:
        if counter > 0:
            # Rows before this don't contain the data
            row = row[0].split(',')
            key = int(row[0])
            value = float(row[1])
            pd_response[key] = value
        counter +=1

# Unpickle the measurements

In [ ]:
# Unpickle the data
filename = "/home/c/cslage/u/AuxTel/filter_data/qe_test_21Dec23.pkl"
file = open(filename, "rb")
results = pkl.load(file)
file.close()

In [ ]:
#Screwed up the zero point current
results['zero_point_current'] = -8.708e-08

In [ ]:
wave = 550
pd_data = -(results[str(wave)]['electrometer_current'] - results['zero_point_current']) / pd_response[wave]
print(pd_data)

# Now process the measured images

In [ ]:
wave_step = 25 # Wavelength step in nm
min_wave = 350; max_wave = 1050;
wave = min_wave
expId = 2023122100021

waves = []
responses = []
pd_datas = []
while wave < max_wave + wave_step / 2.0:
    dataId = {'exposure': expId, 'detector': 0, "instrument": 'LATISS'}
    exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
    data = quickSmooth(exp.image.array, 1)
    pd_data = -(results[str(wave)]['electrometer_current'] - results['zero_point_current']) / pd_response[wave]
    pd_datas.append(pd_data)
    response = np.median(data) / pd_data
    waves.append(wave)
    responses.append(response)
    print(f"At {wave} nm, response is {response}")
    wave += wave_step
    expId += 1


# Now plot the data

In [ ]:
plt.title("AuxTel Test QE curve - 21-Dec-23")
plt.plot(waves, responses, marker='x', color='blue', label="Measured")
#plt.plot(official_waves, official_trans, marker='+', color='red', label="Official")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Response (arbitrary untits)")
plt.xlim(350, 1050)
#plt.ylim(0,100.0)
#plt.legend()
plt.savefig("/home/c/cslage/u/AuxTel/filter_data/QE_Test_Curve_21Dec23.png")

In [ ]:
plt.title("AuxTel Test QE curve - 21-Dec-23")
plt.plot(waves, pd_datas, marker='x', color='blue', label="Measured")
#plt.plot(official_waves, official_trans, marker='+', color='red', label="Official")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Photodiode data current (A)\n normalized by response")
plt.xlim(350, 1050)
#plt.ylim(0,100.0)
#plt.legend()
plt.savefig("/home/c/cslage/u/AuxTel/filter_data/QE_Test_PD_Data_21Dec23.png")